# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучите общую информацию

In [1]:
#Импортируем библиотеку Pandas, Mystem, Counter (последние 2 для лемматизации) открываем наш файл с данными и смотрим общую 
#информацию с типами данных и количеством значений в каждом столбце
import pandas as pd
pd.options.mode.chained_assignment = None
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Можно заметить, что в столбцах "days_employed" и "total_income" (общий трудовой стаж в днях и ежемесячный доход) в некоторых строках отсутствуют значения, при этом тип данных в этих столбцах float64.
В столбце "days_employed" также имеются отрицательные значения. 
С названиями столбцов проблем не обнаружено.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#Посмотрим пропущенные значения в столбцах файла с помощью метода isna() и sum()
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#Теперь посмотрим, совпадают ли пропуски в одних и тех же ячейках столбцов 'days_employed' и 'total_income'
df_nan = df[(df['days_employed'].isna()) & (df['total_income'].isna())]
df_nan

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
#Рассчитаем средний ежемесячный доход для каждого типа занятости с помощью метода groupby() и mean()
df.groupby('income_type')['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

In [6]:
#Запишем функцию, которая заменит пропуски на средний ежемесячный доход для каждого типа занятости
def average_income(row):
    income_type = row['income_type']
    total_income = row['total_income']
    if income_type == 'безработный':
        return 131339.751676
    if income_type == 'в декрете':
        return 53829.130729
    if income_type == 'госслужащий':
        return 170898.309923
    if income_type == 'компаньон':
        return 202417.461462
    if income_type == 'пенсионер':
        return 137127.465690
    if income_type == 'предприниматель':
        return 499163.144947
    if income_type == 'сотрудник':
        return 161380.260488
    return 98201.625314    

In [7]:
#Теперь применим функцию для замены всех пропусков в значениях столбца 'total_income'
df['total_income'] = df['total_income'].fillna(df_nan.apply(average_income, axis=1))

In [8]:
#Проверим данные
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,137127.465690,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,170898.309923,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,137127.465690,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,170898.309923,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,137127.465690,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,202417.461462,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,161380.260488,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,202417.461462,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,161380.260488,строительство жилой недвижимости


In [9]:
#Пропуски в значениях столбца 'days_employed' нас не сильно интересуют, поскольку здесь сложно сказать, сколько дней кто 
#отработал даже в среднем, поэтому можно просто заменить на 0
df['days_employed'] = df['days_employed'].fillna(0)

In [10]:
#Проверим наши данные на наличие пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Строки с пропущенными значениями в столбцах 'days_employed' и 'total_income' совпадают.
Пропуски в столбце 'total_income' можно заменить на средние значения ежемесячного дохода для каждого типа занятости 'income_type', поскольку эти данные могут пригодиться нам позже.
Пропуски в значениях столбца 'days_employed' нас не сильно интересуют, поскольку здесь сложно сказать, сколько дней кто 
отработал даже в среднем, поэтому можно просто заменить на 0. Помимо этого, встречаются достаточно большие значения даже в днях, скорее всего ошибка в вводе данных (возможно, вводили в часах или вообще по другому принципу)
Отсутствующие значения говорят о том, что их не внесли в данные, что странно, поскольку эта информация является достаточно 
важной.

### Замена типа данных

In [11]:
#Изменим тип данных в столбцах 'days_employed' и 'total_income' с помощью метода astype()
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [12]:
#Проверим изменения
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [13]:
#И тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Были преобразованы данные из типа 'float' в тип 'int'.

### Обработка дубликатов

In [14]:
#Посмотрим все уникальные значения для столбца 'children' с помощью unique()
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [15]:
#Посмотрим сколько строк имеют значения -1 и 20
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [16]:
#Посмотрим все уникальные значения для столбца 'dob_years' с помощью unique()
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [17]:
#Посмотрим все уникальные значения для столбца 'education' с помощью unique()
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [18]:
#Преобразуем все значения в столбце 'education' к одному виду с помощью метода str.lower()
df['education'] = df['education'].str.lower()

In [19]:
#Проверим изменения
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [20]:
#Посмотрим все уникальные значения для столбца 'education_id' с помощью unique()
df['education_id'].unique()

array([0, 1, 2, 3, 4])

In [21]:
#Посмотрим все уникальные значения для столбца 'family_status' с помощью unique() 
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [22]:
#Посмотрим все уникальные значения для столбца 'family_status_id' с помощью unique()
df['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [23]:
#Посмотрим все уникальные значения для столбца 'gender' с помощью unique()
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [24]:
#Посмотрим сколько строк имеют значения XNA
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [25]:
#Посмотрим все уникальные значения для столбца 'debt' с помощью unique()
df['debt'].unique()

array([0, 1])

In [26]:
#Посмотрим все уникальные значения для столбца 'purpose' с помощью unique()
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [27]:
#Теперь посмотрим количество дубликатов
df.duplicated().sum()

71

In [28]:
#Удалим все дубликаты
df = df.drop_duplicates()

In [29]:
#Проверим количество дубликатов
df.duplicated().sum()

0

**Вывод**

Приведение к одному виду потребовалось только для столбца 'education'. В столбце 'children' значения -1 скорее всего означают, что данных о количестве детей нет, что странно. Значение в столбце 'children' 20 скорее всего возможно, но слишком много строк с этим значением. Значение 0 в столбце 'dob_years' скорее всего говорит об отсутствии данных о возрасте, что тоже странно. Есть одно значение XNA в столбце 'gender', что скорее всего говорит о том, что пол не указан.
Возможные дубликаты скорее всего были получены вследствие задвоения данных или создали ошибочно повтор.

### Лемматизация

In [30]:
#Посмотрим, сколько значений в каждой категории
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [31]:
#Выделим общие слова из всех имеющихся категорий
categories = ['свадьба', 'недвижимость', 'образование', 'автомобиль', 'жилье']

In [32]:
#Применим лемматизацию
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma
df['purpose_group'] = df['purpose'].apply(lemmatize)     

In [33]:
#Заменим значения, где есть слово 'жилье' на слово 'недвижимость'
df.loc[df['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'

In [34]:
#Подсчитаем получившиеся значения в новых категориях
df['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

**Вывод**

С помощью лемматизации из всех категорий столбца 'purpose' были выведены общие категории в отдельный столбец 'purpose_group'. В категориях предназначения кредита скорее всего по-разному записывали информацию.

### Категоризация данных

In [35]:
#Для начала возьмём все отрицательные значения в столбце 'days_employed' по модулю
df['days_employed'] = abs(df['days_employed'])
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,недвижимость
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,автомобиль
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,автомобиль


In [36]:
#Проверим, каким названиям какой 'id' присваивается и сколько семейных положений с каждым статусом и 'id'
df.groupby(['family_status','family_status_id'])['family_status'].count()

family_status          family_status_id
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
гражданский брак       1                    4151
женат / замужем        0                   12339
Name: family_status, dtype: int64

In [37]:
#Проверим, каким названиям какой 'id' присваивается и сколько разных видов образований и 'id'
df.groupby(['education','education_id'])['education'].count()

education            education_id
высшее               0                5250
начальное            3                 282
неоконченное высшее  2                 744
среднее              1               15172
ученая степень       4                   6
Name: education, dtype: int64

In [38]:
#Далее разделим возраст людей по группам с помощью функции
def human_age(row):
    dob_years = row['dob_years']
    if dob_years <= 30:
        return 'До 30 лет'
    if 30 < dob_years <= 55:
        return 'До 55 лет'
    return 'Старше 55 лет'    

In [39]:
#Разделим людей по ежемесячному доходу с помощью функции
def salary_level(row):
    salary = row['total_income']
    if salary <= 50000:
        return 'Низкий'
    if 50000 < salary <= 100000:
        return 'Средний'
    if 100000 < salary <= 500000:
        return 'Высокий'
    return 'Очень высокий' 

In [40]:
#Разделим людей по количеству детей (поскольку у нас есть значения -1, то для него будет вынесен отдельный пункт с названием 
#'Нет данных')
def child_count(row):
    child_count = row['children']
    if child_count == 0:
        return 'Не имеет детей'
    if 1 <= child_count <= 2:
        return 'Имеет 1-2 ребёнка'
    if child_count >= 3:
        return 'Имеет много детей'
    else:
        return 'Нет данных'

In [41]:
#Теперь применим все эти функции для наших данных
df['age_range'] = df.apply(human_age, axis=1)
df['salary_level'] = df.apply(salary_level, axis=1)
df['child_count_range'] = df.apply(child_count, axis=1)

In [42]:
#Теперь посмотрим как всё это выглядит
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,age_range,salary_level,child_count_range
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,До 55 лет,Высокий,Имеет 1-2 ребёнка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,До 55 лет,Высокий,Имеет 1-2 ребёнка
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,До 55 лет,Высокий,Не имеет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,До 55 лет,Высокий,Имеет много детей
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,До 55 лет,Высокий,Не имеет детей


**Вывод**

Были категоризованы данные по разным группам (возраст, количество детей, уровень зарплаты) с целью дальнейшего получения финальной статистики на основе этих данных. 
Помимо этого, были проверены какие 'id' каким названиям принадлежат для образования и семейного положения (ошибок не обнаружено). 

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий
df.groupby(['child_count_range', 'debt'])['debt'].count()

child_count_range  debt
Имеет 1-2 ребёнка  0        6222
                   1         638
Имеет много детей  0         417
                   1          39
Не имеет детей     0       13028
                   1        1063
Нет данных         0          46
                   1           1
Name: debt, dtype: int64

In [44]:
#Создадим новый датафрейм для нахождения отношения количества людей с "разными категориями детей" на сумму значений с 
#задолженностями (0 - не имеет задолженности, 1 - имеет задолженность)
children_debt = pd.DataFrame()
children_debt['count'] = df.groupby('child_count_range')['debt'].count()
children_debt['sum'] = df.groupby('child_count_range')['debt'].sum()
children_debt['result'] = children_debt['sum'] / children_debt['count']

In [45]:
#Выведим получившуюся информацию на экран
children_debt.sort_values(by='result', ascending=False)

,count,sum,result
child_count_range,,,
Имеет 1-2 ребёнка,6860,638,0.093003
Имеет много детей,456,39,0.085526
Не имеет детей,14091,1063,0.075438
Нет данных,47,1,0.021277


In [46]:
#Сделаем то же самое, но с помощью сводной таблицы и в %
children_debt_pivot_table = df.pivot_table(index='child_count_range', columns='debt', values='children', 
                                           aggfunc='count')
children_debt_pivot_table['result, %'] = round((children_debt_pivot_table[1] / (children_debt_pivot_table[0] 
                                                                            + children_debt_pivot_table[1])) * 100, 1)
children_debt_pivot_table.sort_values(by='result, %', ascending=False)

debt,0,1,"result, %"
child_count_range,,,
Имеет 1-2 ребёнка,6222,638,9.3
Имеет много детей,417,39,8.6
Не имеет детей,13028,1063,7.5
Нет данных,46,1,2.1


**Вывод**

Как можно заметить из получившейся статистики, те, кто имеют 1-2 ребёнка, чуть чаще имеют задолженность, чем те, кто имеет много детей или не имеет их вообще, хоть разница примерно в 1-2 %. По поводу 'Нет данных' сложно дать информацию из-за её отсутствия. Возможно, потребуется большая выборка в дальнейших исследованиях для получения более подробных данных.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий
df.groupby(['family_status', 'debt'])['debt'].count()

family_status          debt
Не женат / не замужем  0        2536
                       1         274
в разводе              0        1110
                       1          85
вдовец / вдова         0         896
                       1          63
гражданский брак       0        3763
                       1         388
женат / замужем        0       11408
                       1         931
Name: debt, dtype: int64

In [48]:
#Создадим новый датафрейм для нахождения отношения количества людей с "разным семейным положением" на сумму значений с 
#задолженностями (0 - не имеет задолженности, 1 - имеет задолженность)
family_debt = pd.DataFrame()
family_debt['count'] = df.groupby('family_status')['debt'].count()
family_debt['sum'] = df.groupby('family_status')['debt'].sum()
family_debt['result'] = family_debt['sum'] / family_debt['count']

In [49]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий по убыванию столбца "result"
family_debt.sort_values(by='result', ascending=False)

,count,sum,result
family_status,,,
Не женат / не замужем,2810,274,0.097509
гражданский брак,4151,388,0.093471
женат / замужем,12339,931,0.075452
в разводе,1195,85,0.071130
вдовец / вдова,959,63,0.065693


In [50]:
#Сделаем то же самое, но с помощью сводной таблицы и в %
family_debt_pivot_table = df.pivot_table(index='family_status', columns='debt', values='children', 
                                           aggfunc='count')
family_debt_pivot_table['result, %'] = round((family_debt_pivot_table[1] / (family_debt_pivot_table[0] 
                                                                            + family_debt_pivot_table[1])) * 100, 1)
family_debt_pivot_table.sort_values(by='result, %', ascending=False)

debt,0,1,"result, %"
family_status,,,
Не женат / не замужем,2536,274,9.8
гражданский брак,3763,388,9.3
женат / замужем,11408,931,7.5
в разводе,1110,85,7.1
вдовец / вдова,896,63,6.6


**Вывод**

Те, кто не женат/не замужем и имеют гражданский брак, реже возвращают задолженность вовремя, чем те, кто женат/замужем, в разводе или вдовец/вдова (с последними разница аж в ~3 %) 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [51]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий
df.groupby(['salary_level', 'debt'])['debt'].count()

salary_level   debt
Высокий        0       15396
               1        1373
Низкий         0         349
               1          23
Очень высокий  0         208
               1          14
Средний        0        3760
               1         331
Name: debt, dtype: int64

In [52]:
#Создадим новый датафрейм для нахождения отношения количества людей с разным уровнем дохода на сумму значений с 
#задолженностями (0 - не имеет задолженности, 1 - имеет задолженность)
salary_debt = pd.DataFrame()
salary_debt['count'] = df.groupby('salary_level')['debt'].count()
salary_debt['sum'] = df.groupby('salary_level')['debt'].sum()
salary_debt['result'] = salary_debt['sum'] / salary_debt['count']

In [53]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий по убыванию столбца "result"
salary_debt.sort_values(by='result', ascending=False)

,count,sum,result
salary_level,,,
Высокий,16769,1373,0.081877
Средний,4091,331,0.080909
Очень высокий,222,14,0.063063
Низкий,372,23,0.061828


In [54]:
#Сделаем то же самое, но с помощью сводной таблицы и в %
salary_debt_pivot_table = df.pivot_table(index='salary_level', columns='debt', values='children', 
                                           aggfunc='count')
salary_debt_pivot_table['result, %'] = round((salary_debt_pivot_table[1] / (salary_debt_pivot_table[0] 
                                                                            + salary_debt_pivot_table[1])) * 100, 1)
salary_debt_pivot_table.sort_values(by='result, %', ascending=False)

debt,0,1,"result, %"
salary_level,,,
Высокий,15396,1373,8.2
Средний,3760,331,8.1
Очень высокий,208,14,6.3
Низкий,349,23,6.2


**Вывод**

Те, у кого высокий и средний уровни зарплаты, имеют больше задолженностей, чем те, у кого очень высокий и низкий уровни зарплаты (разница примерно в ~2 %). Возможно, потребуется большая выборка, поскольку были взяты различные диапазоны зарплат (для низкой <= 50000, для средней 50000 - 100000, для высокой 100000 - 500000, для очень высокой > 500000)

- Как разные цели кредита влияют на его возврат в срок?

In [55]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий
df.groupby(['purpose_group', 'debt'])['debt'].count()

purpose_group  debt
автомобиль     0        3903
               1         403
недвижимость   0       10029
               1         782
образование    0        3643
               1         370
свадьба        0        2138
               1         186
Name: debt, dtype: int64

In [56]:
#Создадим новый датафрейм для нахождения отношения количества людей с разными целями кредита на сумму значений с 
#задолженностями (0 - не имеет задолженности, 1 - имеет задолженность)
purpose_debt = pd.DataFrame()
purpose_debt['count'] = df.groupby('purpose_group')['debt'].count()
purpose_debt['sum'] = df.groupby('purpose_group')['debt'].sum()
purpose_debt['result'] = purpose_debt['sum'] / purpose_debt['count']

In [57]:
#Посмотрим все значения с наличием или отсутствием задолженности для разных категорий по убыванию столбца "result"
purpose_debt.sort_values(by='result', ascending=False)

,count,sum,result
purpose_group,,,
автомобиль,4306,403,0.093590
образование,4013,370,0.092200
свадьба,2324,186,0.080034
недвижимость,10811,782,0.072334


In [58]:
#Сделаем то же самое, но с помощью сводной таблицы и в %
purpose_debt_pivot_table = df.pivot_table(index='purpose_group', columns='debt', values='children', 
                                           aggfunc='count')
purpose_debt_pivot_table['result, %'] = round((purpose_debt_pivot_table[1] / (purpose_debt_pivot_table[0] 
                                                                            + purpose_debt_pivot_table[1])) * 100, 1)
purpose_debt_pivot_table.sort_values(by='result, %', ascending=False)

debt,0,1,"result, %"
purpose_group,,,
автомобиль,3903,403,9.4
образование,3643,370,9.2
свадьба,2138,186,8.0
недвижимость,10029,782,7.2


**Вывод**

Те, кто имели задолженности по кредиту на автомобили и образование, имеют на 1-2 % больше вероятность не погасить задолженность вовремя, чем те, кто брал кредит на свадьбу и недвижимость. Опять же, скорее всего нужна более крупная выборка, поскольку неизвестно, на какую машину, какое образование и какую недвижимость брали деньги.

## Шаг 4. Общий вывод

Общий вывод сделать достаточно сложно, поскольку требуются дополнительные исследования во многих пунктах. Например, много непонятных значений в разных столбцах и неизвестно, как и по какому принципу они заполнялись. Для нахождения завимости между разными категориями в большинстве случаев потребуется большая выборка. В целом: те, кто имеют 1-2 ребёнка, чуть чаще имеют задолженность, чем те, кто имеет много детей или не имеет их вообще, хоть разница примерно в 1-2 %; те, кто не женат/не замужем и имеют гражданский брак, реже возвращают задолженность вовремя, чем те, кто женат/замужем, в разводе или вдовец/вдова (с последними разница аж в ~3 %); те, у кого высокий и средний уровни зарплаты, имеют больше задолженностей, чем те, у кого очень высокий и низкий уровни зарплаты (разница примерно в ~2 %); те, кто имели задолженности по кредиту на автомобили и образование, имеют на 1-2 % больше вероятность не погасить задолженность вовремя, чем те, кто брал кредит на свадьбу и недвижимость.